<a href="https://colab.research.google.com/github/criamadei/Lunar-Mars-Crater/blob/main/Lunar_Mars_Crater.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Lunar and Martian Crater Detection with YOLOv8

This project demonstrates the use of YOLOv8 (You Only Look Once, version 8) for detecting craters on Martian and Lunar surfaces. Leveraging a dataset specifically curated for crater identification, the model is trained to accurately pinpoint these geological features, which are crucial for planetary science, landing site selection, and understanding celestial body evolution.

### Project Workflow:

1.  **Environment Setup**: Installation of necessary libraries, including `ultralytics` for YOLOv8.
2.  **Hardware Check**: Verification of GPU availability to accelerate training.
3.  **Data Acquisition**: Downloading and extraction of the "Martian Lunar Crater Detection Dataset" from Kaggle.
4.  **Dataset Configuration**: Creation and refinement of a `craters.yaml` file to properly map dataset paths for YOLOv8.
5.  **Model Training**: Training a YOLOv8s model with custom hyperparameters for improved detection performance.
6.  **Results Analysis**: Visualization of training metrics (loss curves, precision, recall) and confusion matrix.
7.  **Visual Inference**: Demonstrating the trained model's detection capabilities on sample images.
8.  **Model Export**: Providing the option to download the best-trained model weights.
9.  **GIF Generation**: Creating an animated GIF to visually showcase the model's real-time detection in action.

This notebook provides a complete pipeline from data preparation to model deployment and visualization for an object detection task in a specialized domain.

In [ ]:
%%capture
# ==========================================
# 1. CONFIGURATION
# ==========================================
!pip install -q --upgrade pip
!pip install -q pandas numpy matplotlib seaborn scikit-learn ultralytics


In [ ]:
# ==========================================
# 2. SETUP AND IMPORT
# ==========================================

import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ultralytics import YOLO

# Graph style configuration (Pro-tip: makes everything more readable)
sns.set_theme(style="whitegrid", context="notebook")
plt.rcParams['figure.figsize'] = (10, 6) # Set standard dimension for graph
%matplotlib inline

print("✅ Libraries imported and graph style configured.")

In [ ]:
# ==========================================
# 3. HARDWARE CHECK (GPU)
# ==========================================
import torch

if torch.cuda.is_available():
    print(f"✅ GPU Found: {torch.cuda.get_device_name(0)}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ Warning: GPU not found. The code will run on CPU (slower).")
    print("   Tip: Go to Runtime > Change runtime type > T4 GPU") #in Colab settings

In [ ]:
# ==========================================
# 4. SETTING SEED
# ==========================================

def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    print(f"✅ Random seed set to: {seed}")

set_seed(42)

In [ ]:
# ==========================================
# 5. KAGGLE AUTH (SECURE)
# ==========================================
import os
from google.colab import userdata

try:
    os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')
    os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')
    print("✅ Kaggle credential loaded from Secrets.")
except Exception as e:
    print("❌ Error: Set 'KAGGLE_USERNAME' and 'KAGGLE_KEY' in Colab Secrets.")

In [ ]:
# ==========================================
# 6. DOWNLOAD DATASET (Mars/Moon Object Detection)
# ==========================================
import os

dataset_slug = "lincolnzh/martianlunar-crater-detection-dataset"

# Download
if not os.path.exists('martianlunar-crater-detection-dataset.zip'):
    print(f"⬇️ Downloading {dataset_slug}...")
    !kaggle datasets download -d $dataset_slug
    !unzip -q martianlunar-crater-detection-dataset.zip -d dataset_mars_moon
    print("✅ Download done.")
else:
    print("✅ Dataset already present.")

In [ ]:
# ==========================================
# 7. CONFIGURAZIONE DATASET (YAML)
# ==========================================
import yaml

# Define the absolute path where we extracted the data
base_dir = '/content/dataset_mars_moon'

# Try to understand the internal structure (often Kaggle datasets have subfolders 'images' or 'martian_lunar...')
# For this specific dataset, the structure usually has mixed images and labels or separate folders.
# Create a yaml file that points to the correct folders.

# YOLO expects this structure in the yaml file:
data_config = {
    'path': base_dir,         # Root folder
    'train': 'images',        # Training images subfolder (or relative path)
    'val': 'images',          # Use the same for validation if there's no separate folder (or do split)
    'names': {0: 'Crater'}    # Class name
}

# IMPORTANT NOTE:
# Often this specific LincolnZh dataset has a somewhat disorganized structure.
# If you see "No images found" errors, you might need to move the files.
# This standard procedure tries to configure it:

with open('craters.yaml', 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False)

print("✅ File 'craters.yaml' created. YOLO will know where to search.")
print("⚠️ WARNING: If training fails, check the 'dataset_mars_moon' folder on the left.")

In [ ]:
%%capture
# ==========================================
# 8. TRAINING YOLOv8
# ==========================================
from ultralytics import YOLO
import yaml # Import yaml to re-create the config file
import os   # Import os to use base_dir

# 1. Load a pre-trained model (Nano = very fast, Small = more accurate)
model = YOLO('yolov8s.pt')  # Automatically downloads the initial weights

# --- FIX START ---
# The original craters.yaml was configured incorrectly.
# Redefine the base directory and correct the paths for images and labels.
base_dir = '/content/dataset_mars_moon'

data_config = {
    'path': os.path.join(base_dir, 'craters'), # The root for train/val/test is inside 'craters' folder
    'train': 'train/images',
    'val': 'valid/images',
    'test': 'test/images', # It's good practice to include the test set path if available
    'names': {0: 'Crater'}
}

# Overwrite the existing 'craters.yaml' with the corrected paths
with open('craters.yaml', 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False)

print("✅ File 'craters.yaml' updated with correct paths.")
# --- FIX END ---

# 2. Start training
# epochs=20: Number of epochs
# imgsz=640: Image size (YOLO standard)
# data='craters.yaml': The file we created above
print("🚀 Starting YOLOv8 training...")

results = model.train(
    data='craters.yaml',
    epochs=100,
    patience=10,
    imgsz=1240,
    batch=16,
    dropout=0.3,
    freeze=10,

    degrees=180.0,
    flipud=0.5,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.1,

    optimizer='AdamW',
    lr0=0.001,
    cos_lr=True,
    project='lunar_crater_project',
    name='yolo_run',
    exist_ok=True
)

print("✅ Training completed!")

In [ ]:
# ==========================================
# 8-BIS. TRAINING COMPLETION CONFIRMATION
# ==========================================
print("✅ Training completed in background!")
print(f"Results are saved in: {results.save_dir}")

# Show the final result (mAP) immediately to see how it went
metrics = model.val() # Calculate final metrics
print(f"\n🎯 Final mAP50: {metrics.box.map50:.4f}")
print(f"🎯 Final mAP50-95: {metrics.box.map:.4f}")

### Results Analysis Summary

The training process was successfully completed. Below, you can observe the learning curves for various metrics (Loss, Precision, Recall) and the Confusion Matrix generated during the training of the YOLOv8 model for crater detection.

*   **Loss Curves**: These graphs illustrate how the model's error (loss) decreased over epochs, indicating that the model was learning and converging.
*   **Precision and Recall Curves**: These show the trade-off between precision (accuracy of positive predictions) and recall (ability to find all positive samples) as training progressed. Ideally, both should increase and stabilize at high values.
*   **Confusion Matrix**: This visualizes the performance of the classification model, showing true positives, true negatives, false positives, and false negatives. For single-class detection like 'Crater', it primarily indicates how well craters are detected versus background.

The overall metrics (mAP50, mAP50-95) provide a quantitative measure of the model's accuracy in detecting craters. Further analysis of these plots can help identify potential areas for improvement, such as adjusting hyperparameters or augmenting the dataset.

In [ ]:
# ==========================================
# 9. RESULT ANALYSIS
# ==========================================
from IPython.display import Image, display

# Path generated by YOLO (based on project='lunar_crater_project' and name='yolo_run')
# If you changed the names in the previous cell, update here too.
results_path = 'lunar_crater_project/yolo_run/results.png'
confusion_matrix_path = 'lunar_crater_project/yolo_run/confusion_matrix.png'

print("📊 Learning curves (Loss, Precision, Recall):")
try:
    display(Image(filename=results_path, width=800))
except:
    print("⚠️ Graphic results.png not found. Check the directory.")

print("\n🧩 Confusion Matrix (How much the model gets confused):")
try:
    display(Image(filename=confusion_matrix_path, width=600))
except:
    print("⚠️ Confusion matrix not found.")

In [ ]:
# ==========================================
# 10. TEST VISIVO (INFERENZA)
# ==========================================
import glob
import random
import matplotlib.pyplot as plt
import cv2

# Take 3 random images from the test/validation folder
# Note: Adjust the path if your images are in a different subfolder
test_images = glob.glob('dataset_mars_moon/**/*.jpg', recursive=True)
# Take a random subset
sample_images = random.sample(test_images, 3)

# Load the best model obtained from training
best_model = YOLO('lunar_crater_project/yolo_run/weights/best.pt')

plt.figure(figsize=(15, 5))

for i, img_path in enumerate(sample_images):
    # Execute the prediction
    results = best_model.predict(source=img_path, conf=0.25, verbose=False) # conf=0.25 is the confidence threshold

    # YOLO has an internal function to draw boxes
    res_plotted = results[0].plot()

    # Convert from BGR (OpenCV) to RGB (Matplotlib) to see it with correct colors
    res_plotted = cv2.cvtColor(res_plotted, cv2.COLOR_BGR2RGB)

    plt.subplot(1, 3, i + 1)
    plt.imshow(res_plotted)
    plt.axis('off')
    plt.title(f"Test Image {i+1}")

plt.tight_layout()
plt.show()

In [ ]:
# ==========================================
# 11. EXPORT DEL MODELLO
# ==========================================
from google.colab import files

try:
    print("⬇️ Downloading the best model (best.pt)...")
    files.download('lunar_crater_project/yolo_run/weights/best.pt')
except Exception as e:
    print("Could not download the file automatically.")
    print("You can find it on the left at: lunar_crater_project/yolo_run/weights/best.pt")

In [ ]:
# ==========================================
# 12. GIF GENERATOR
# ==========================================
import imageio
import glob
import random
import cv2
import os
from IPython.display import Image, display

print("🎬 Creazione della GIF dimostrativa...")

# 1. Load the best model
# Make sure the path is correct (you can find it in the folder on the left)
model_path = 'lunar_crater_project/yolo_run/weights/best.pt'

# 2. Take 15 random images from the dataset
all_images = glob.glob('dataset_mars_moon/**/*.jpg', recursive=True)
selected_images = random.sample(all_images, 15)

frames = []

for img_path in selected_images:
    # Execute the prediction
    results = model.predict(img_path, conf=0.25, verbose=False)

    # Draw boxes on the image
    res_plotted = results[0].plot()

    # Convert colors from BGR (OpenCV) to RGB (Standard)
    res_plotted = cv2.cvtColor(res_plotted, cv2.COLOR_BGR2RGB)

    # Add to the frames list
    frames.append(res_plotted)

# 3. Save the GIF
gif_path = "crater_detection_demo.gif"
# duration=500 means 500ms per frame (2 frames per second), loop=0 is infinite
imageio.mimsave(gif_path, frames, duration=500, loop=0)

print(f"✅ GIF salvata come: {gif_path}")

# 4. Display it here immediately
display(Image(filename=gif_path, width=600))